In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')

from keras.layers.normalization import BatchNormalization
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report

In [2]:
#Data Preparation

df_image = pd.read_csv('./image_info/train_image_level.csv')
df_study = pd.read_csv('./image_info/train_study_level.csv')
df_study['id'] = df_study['id'].str.replace('_study',"")
df_study.rename({'id': 'StudyInstanceUID'},axis=1, inplace=True)
df_train = df_image.merge(df_study, on='StudyInstanceUID')
df_train.loc[df_train['Negative for Pneumonia']==1, 'study_label'] = 'negative'
df_train.loc[df_train['Typical Appearance']==1, 'study_label'] = 'typical'
df_train.loc[df_train['Indeterminate Appearance']==1, 'study_label'] = 'indeterminate'
df_train.loc[df_train['Atypical Appearance']==1, 'study_label'] = 'atypical'
df_train.drop(['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], axis=1, inplace=True)
df_train['id'] = df_train['id'].str.replace('_image', '.jpg')
df_train['image_label'] = df_train['label'].str.split().apply(lambda x : x[0])
df_size = pd.read_csv('./image_info/size.csv')
df_train = df_train.merge(df_size, on='id')
df_train.head(3)

,id,boxes,label,StudyInstanceUID,study_label,image_label,dim0,dim1,split
0,000a312787f2.jpg,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,typical,opacity,3488,4256,train
1,000c3a3f293f.jpg,NaN,none 1 0 0 1 1,ff0879eb20ed,negative,none,2320,2832,train
2,0012ff7358bc.jpg,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,typical,opacity,2544,3056,train


In [3]:
img_size = 224
batch_size = 64

# Get the data generator for different image preprocessing technics
def preprocess_image(img):
    equ_img = exposure.equalize_adapthist(img/255, clip_limit=0.05, kernel_size=24)
    return equ_img

def get_data_generators(name, CLAHE = False):
    
    image_generator = ImageDataGenerator(
        validation_split=0.2,
        horizontal_flip = True,
        zoom_range = 0.15,
        rotation_range = 10,
        brightness_range = [0.8, 1.2],
        fill_mode='nearest'
    )

    image_generator_valid = ImageDataGenerator(validation_split=0.2)

    train_generator = image_generator.flow_from_dataframe(
            dataframe = df_train,
            directory='./images/512' + name + '/train',
            x_col = 'id',
            y_col =  'study_label',  
            target_size=(img_size, img_size),
            batch_size=batch_size,
            subset='training', seed = 23,
            class_mode='categorical',
            preprocessing_function = preprocess_image if CLAHE else None) 

    valid_generator=image_generator_valid.flow_from_dataframe(
        dataframe = df_train,
        directory='./images/512' + name + '/train',
        x_col = 'id',
        y_col = 'study_label',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        subset='validation', shuffle=False, seed=23,
        class_mode='categorical',
        preprocessing_function = preprocess_image if CLAHE else None)
    
    return(train_generator, valid_generator)

# Get the untrained model
def get_model():
    pre_model = DenseNet121(weights=None, include_top=False, input_shape=(img_size,img_size,3))
    out = Dense(64, activation='sigmoid')(pre_model.output)
    pre_model = Model(inputs=pre_model.input, outputs=out)
    x = pre_model.layers[-2].output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    output = Dense(4, activation='softmax')(x)
    model = Model(pre_model.input, output)
    return(model)

# Report the accuracy and other metrics
def validate_model(model, valid_generator):
    predictions = model.predict_generator(valid_generator, 80)
    truth = valid_generator.classes
    result = np.argmax(predictions, axis = 1)
    print(classification_report(truth, result))

In [4]:
# Images Training without Image Enhancements

model = get_model()
train_generator, valid_generator = get_data_generators("jpg")
model.compile(optimizer = "rmsprop", loss='categorical_crossentropy', metrics='accuracy')

history = model.fit(
      train_generator,
      epochs=1,
      validation_data=valid_generator,
      verbose=1)

validate_model(model, valid_generator)
model.save("./models/dense121_naive.h5")

Found 5068 validated image filenames belonging to 4 classes.
Found 1266 validated image filenames belonging to 4 classes.
80/80 [==============================] - 1843s 23s/step - loss: 1.2868 - accuracy: 0.4637 - val_loss: 58555.6094 - val_accuracy: 0.4597
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        89
           1       0.00      0.00      0.00       239
           2       0.00      0.00      0.00       356
           3       0.46      1.00      0.63       582

    accuracy                           0.46      1266
   macro avg       0.11      0.25      0.16      1266
weighted avg       0.21      0.46      0.29      1266



In [5]:
# Images Training with High-frequency emphasis filtering

model = get_model()
train_generator, valid_generator = get_data_generators("hef")
model.compile(optimizer = "rmsprop", loss='categorical_crossentropy', metrics='accuracy')

history = model.fit(
      train_generator,
      epochs=1,
      validation_data=valid_generator,
      verbose=1)

validate_model(model, valid_generator)
model.save("./models/dense121_hef.h5")

Found 5068 validated image filenames belonging to 4 classes.
Found 1266 validated image filenames belonging to 4 classes.
80/80 [==============================] - 1494s 18s/step - loss: 1.2912 - accuracy: 0.4655 - val_loss: 2636.4927 - val_accuracy: 0.4597
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        89
           1       0.00      0.00      0.00       239
           2       0.00      0.00      0.00       356
           3       0.46      1.00      0.63       582

    accuracy                           0.46      1266
   macro avg       0.11      0.25      0.16      1266
weighted avg       0.21      0.46      0.29      1266



In [6]:
# Images Training with Unsharp Masking

model = get_model()
train_generator, valid_generator = get_data_generators("um")
model.compile(optimizer = "rmsprop", loss='categorical_crossentropy', metrics='accuracy')

history = model.fit(
      train_generator,
      epochs=1,
      validation_data=valid_generator,
      verbose=1)

validate_model(model, valid_generator)
model.save("./models/dense121_um.h5")

Found 5068 validated image filenames belonging to 4 classes.
Found 1266 validated image filenames belonging to 4 classes.
80/80 [==============================] - 1649s 20s/step - loss: 1.3268 - accuracy: 0.4586 - val_loss: 2894.1199 - val_accuracy: 0.2820
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        89
           1       0.25      0.00      0.01       239
           2       0.28      1.00      0.44       356
           3       0.00      0.00      0.00       582

    accuracy                           0.28      1266
   macro avg       0.13      0.25      0.11      1266
weighted avg       0.13      0.28      0.13      1266



In [7]:
# Images Trainning with Contrast Limited Adaptive Histogram Equalization

model = get_model()
train_generator, valid_generator = get_data_generators("jpg", CLAHE = True)
model.compile(optimizer = "rmsprop", loss='categorical_crossentropy', metrics='accuracy')

history = model.fit(
      train_generator,
      epochs=1,
      validation_data=valid_generator,
      verbose=1)

validate_model(model, valid_generator)
model.save("./models/dense121_clahe.h5")

Found 5068 validated image filenames belonging to 4 classes.
Found 1266 validated image filenames belonging to 4 classes.
80/80 [==============================] - 1703s 21s/step - loss: 1.2902 - accuracy: 0.4542 - val_loss: 23678.5234 - val_accuracy: 0.4589
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        89
           1       0.00      0.00      0.00       239
           2       0.00      0.00      0.00       356
           3       0.46      1.00      0.63       582

    accuracy                           0.46      1266
   macro avg       0.11      0.25      0.16      1266
weighted avg       0.21      0.46      0.29      1266

